In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + "049f4efb51f94ddd9c2c52f781bf3567"
print (url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=049f4efb51f94ddd9c2c52f781bf3567


In [3]:
lats = np.random.uniform(low=-90.000, high=90.000, size=10)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10)
lat_lngs = zip(lats, lngs)

In [4]:
coordinates = list(lat_lngs)

In [5]:
cities = []
for coordinate in coordinates:
    
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if  city not in cities:
    
        cities.append(city)

In [6]:
print (cities)

['tuktoyaktuk', 'egvekinot', 'illoqqortoormiut', 'east london', 'kapaa', 'yellowknife', 'kataysk', 'mantua', 'bokovskaya', 'leua']


In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [8]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        if(i==1):
            print(city_url)
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
#         if(i==1):
#             print(city_url)
#             print(city_weather)
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        des = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Discription": des})
                          #"Date": city_date })

    # If an error is experienced, skip the city.
    except:
            print("City not found. Skipping...")
            pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | tuktoyaktuk
Processing Record 2 of Set 1 | egvekinot
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=049f4efb51f94ddd9c2c52f781bf3567&q=egvekinot
Processing Record 3 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 4 of Set 1 | east london
Processing Record 5 of Set 1 | kapaa
Processing Record 6 of Set 1 | yellowknife
Processing Record 7 of Set 1 | kataysk
Processing Record 8 of Set 1 | mantua
Processing Record 9 of Set 1 | bokovskaya
Processing Record 10 of Set 1 | leua
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
city_data_df = pd.DataFrame(city_data)
city_data_df. head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Discription
0,Tuktoyaktuk,69.4541,-133.0374,-2.20,65,1,8.05,CA,clear sky
1,Egvekinot,66.3167,-179.1667,0.39,67,100,21.61,RU,overcast clouds
2,East London,-33.0153,27.9116,70.61,84,93,3.36,ZA,overcast clouds
3,Kapaa,22.0752,-159.3190,75.20,60,90,16.11,US,overcast clouds
4,Yellowknife,62.4560,-114.3525,9.00,72,90,4.61,CA,light snow
5,Kataysk,56.2890,62.5841,43.02,94,100,6.24,RU,light rain
6,Mantua,45.1667,10.7833,50.00,92,100,1.99,IT,overcast clouds
7,Bokovskaya,49.2290,41.8306,48.99,84,60,9.31,RU,broken clouds
8,Leua,48.2156,18.6071,48.99,67,97,4.00,SK,overcast clouds


In [10]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed","Current Discription"]
city_data_df = city_data_df[new_column_order]
city_data_df 

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Discription
0,Tuktoyaktuk,CA,69.4541,-133.0374,-2.20,65,1,8.05,clear sky
1,Egvekinot,RU,66.3167,-179.1667,0.39,67,100,21.61,overcast clouds
2,East London,ZA,-33.0153,27.9116,70.61,84,93,3.36,overcast clouds
3,Kapaa,US,22.0752,-159.3190,75.20,60,90,16.11,overcast clouds
4,Yellowknife,CA,62.4560,-114.3525,9.00,72,90,4.61,light snow
5,Kataysk,RU,56.2890,62.5841,43.02,94,100,6.24,light rain
6,Mantua,IT,45.1667,10.7833,50.00,92,100,1.99,overcast clouds
7,Bokovskaya,RU,49.2290,41.8306,48.99,84,60,9.31,broken clouds
8,Leua,SK,48.2156,18.6071,48.99,67,97,4.00,overcast clouds


In [11]:
# Create the output file (CSV).
output_data_file = "cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")